In [1]:
import selenium 
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np

In [2]:
df = pd.read_csv('../Data/castilla_leon/poblacion/2862.csv',sep=';')

In [3]:
df = df[(df['Periodo']==2021)&(df['Sexo']=='Total')].reset_index(drop=True)[1:]

In [4]:
df.Municipios = df.Municipios.apply(lambda x: x[5:].strip())

In [5]:
def cambio_nombro(s):
    if ',' in s:
        lli = [i.strip() for i in s.split(',')]
        return lli[1]+' '+lli[0]
    else: return s

In [6]:
df.Municipios = df.Municipios.apply(cambio_nombro)

In [7]:
df.shape

(371, 4)

In [8]:
def scrap_wiki(nombre_pueblo, provincia='Burgos'):
    try:
        URL = f"https://es.wikipedia.org/wiki/{nombre_pueblo.replace(' ','_')}"
        r = requests.get(URL)
        print(r)
        #print(URL)
        soup = BeautifulSoup(r.content, 'lxml') # If this line causes an error, run 'pip install html5lib' or install html5lib
        bodies = soup.findAll('tbody')
        for e in bodies:
            if 'Alcalde' in e.text:
                r_tabla = e.findAll('tr')
        #print(r_tabla)
        for i in r_tabla:
            if 'Alcalde' in i.text:
                alcalde = list(i)[1].text.strip('\n')
                if '(' in alcalde:
                    lli = alcalde.split('(')
                    alcalde = lli[0].strip()
                    partido_pol = lli[1].strip(')')
        return alcalde, partido_pol
    except:
        print(nombre_pueblo)
        return np.nan, np.nan

In [48]:
scrap_wiki('Arandilla')

Arandilla


(nan, nan)

In [46]:
df['new_col'] = df.Municipios.apply(scrap_wiki)

Los Altos
Arandilla
Arcos
Arlanzón
Atapuerca
Baños de Valdearados
Carazo
Cavia
Frías
La Gallega
Grijalba
Haza
Hontanas
Iglesias
Lerma
Milagros
Quintanilla Vivar
Las Quintanillas
Rábanos
Rabé de las Calzadas
Retuerta
Rojas
Rucandio
San Juan del Monte
Santa Cecilia
Santa Inés
Santo Domingo de Silos
Tejada
Valmala
Villafruela
Villegas


In [47]:
df.head()

,Municipios,Sexo,Periodo,Total,new_col
1,Abajas,Total,2021,29.0,"(María Amparo Rodríguez Arnaiz, PP)"
2,Adrada de Haza,Total,2021,197.0,"(María Luisa Plaza Lazar, C's)"
3,Aguas Cándidas,Total,2021,56.0,"(Justo García Bárcena, PP)"
4,Aguilar de Bureba,Total,2021,52.0,"(José María Martínez González, PP)"
5,Albillos,Total,2021,221.0,"(Jesús Ángel Santamaría Martínez, PP)"


In [27]:
df_new_ = pd.DataFrame(df_new)

In [41]:
df_new_['nombre']= df_new_.Municipios.apply(lambda x: x[0])
df_new_['ppolitico']= df_new_.Municipios.apply(lambda x: x[1])


In [45]:
df_new_.ppolitico.value_counts()

PP                                                            194
PSOE                                                           79
C's                                                            22
Cs                                                              7
PCAS-TC                                                         2
Vox                                                             2
Imagina Burgos                                                  2
PR                                                              1
Agrupación Palanciana Independiente                             1
Partido Popular                                                 1
EH Bildu                                                        1
PCAS)[1]​                                                       1
A.G.E.I.S.M.R.                                                  1
AIRE                                                            1
AEI                                                             1
IMC       